In [ ]:
!nvidia-smi

Sat Jun 18 00:03:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers
!pip install simpletransformers

#Setup and Config

In [ ]:
!gdown  1_Zds0QWmbn7bYn3iGrS1gyTLv33ZmwzU

Downloading...
From: https://drive.google.com/uc?id=1_Zds0QWmbn7bYn3iGrS1gyTLv33ZmwzU
To: /content/Memes-final.zip
100% 143M/143M [00:00<00:00, 251MB/s]


In [ ]:
!unzip "/content/Memes-final.zip"
!rm Memes-final.zip

In [ ]:
!mv Memes\ \(final\) Memes-final

In [ ]:
import sklearn
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import seaborn as sns

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
seed = 41

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

# Data

In [ ]:
df = pd.read_excel('/content/Memes-final/final.xlsx')
df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)

In [ ]:
df

,file_name,meme_type,hate_type,meme_type2,hate_type2,text
0,img24.jpg,hate_speech,ddd,0.0,1.0,انا نظري ضعيف ومش هخش الجيش اقسم بالله انت قناص
1,img28.jpg,hate_speech,gender,0.0,4.0,لما فيمينست تتابع هانتر وتعرف غون بيبحث ابوه و...
2,img10.jpg,hate_speech,gender,0.0,4.0,لا أحد النسويات
3,img7.jpg,hate_speech,gender,0.0,4.0,يعني لو قلعنا الحجاب وتعرينا هنطلع القمر أوماااال
4,img16.jpg,hate_speech,gender,0.0,4.0,فيمينيست عربية نفسي اطلع اوروبا حرية وتحرر و ح...
...,...,...,...,...,...,...
993,img1311.jpeg,hate_speech,ddd,0.0,1.0,برو لماذا ابدو كمكعب العشب في ماينكرافت
994,img1313.jpg,hate_speech,ddd,0.0,1.0,حيوانات مصابة بمتلازمة داون المنغولية
995,img1309.png,hate_speech,ddd,0.0,1.0,كنت غبي ولاتستطيع الاصطفاف ابق في المنزل
996,img1308.png,hate_speech,ddd,0.0,1.0,عندما تضغط على ايقاف الفلاش على الكاميرا خاصتك


In [ ]:
train_images = df.file_name
train_text = df.text
train_labels = df.hate_type2

In [ ]:
train_data = []

for i in range(len(df)):
  train_data.append([train_images[i], train_text[i], train_labels[i]])

In [ ]:
train_data = pd.DataFrame(train_data, columns=['images','text','labels'])
train_data

,images,text,labels
0,img24.jpg,انا نظري ضعيف ومش هخش الجيش اقسم بالله انت قناص,1.0
1,img28.jpg,لما فيمينست تتابع هانتر وتعرف غون بيبحث ابوه و...,4.0
2,img10.jpg,لا أحد النسويات,4.0
3,img7.jpg,يعني لو قلعنا الحجاب وتعرينا هنطلع القمر أوماااال,4.0
4,img16.jpg,فيمينيست عربية نفسي اطلع اوروبا حرية وتحرر و ح...,4.0
...,...,...,...
993,img1311.jpeg,برو لماذا ابدو كمكعب العشب في ماينكرافت,1.0
994,img1313.jpg,حيوانات مصابة بمتلازمة داون المنغولية,1.0
995,img1309.png,كنت غبي ولاتستطيع الاصطفاف ابق في المنزل,1.0
996,img1308.png,عندما تضغط على ايقاف الفلاش على الكاميرا خاصتك,1.0


In [ ]:
train_df, test_df = train_test_split(train_data, test_size=0.1, random_state=seed)
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [ ]:
print(len(train_df))
print(len(test_df))

746
93


In [ ]:
train_df

,images,text,labels
985,img1285.png,عندما تعود للمنزل بعدما التنمر عليك لأنك اعمى,1.0
158,img170.jpg,علما تحاصرك الشرطة الأمريكية ثم تتذكر أنك أفغاني,2.0
707,img1033.jpeg,انا لست عنصري ظلي اسود,3.0
876,img1199.jpg,كل هؤلاء الاناس البيض يجعلوني اشتاق لهيروشيما,3.0
681,img1003.jpg,عندما يوم الناس بالتثاؤب هل يظن الاطرش انهم يص...,1.0
...,...,...,...
80,img99.jpg,لما طلاب التحفيظ يطلبوا رحلة للمسبح المدرس الا...,0.0
994,img1313.jpg,حيوانات مصابة بمتلازمة داون المنغولية,1.0
908,img1210.jpg,عندما تقوم بضرب زوجتك لإجبارها على الاجهاض وتر...,3.0
931,img1274.png,عندما يقوم الفتى المعاق بلمسك,1.0


In [ ]:
test_df

,images,text,labels
218,img240.jpg,وأخيرا السعودية تسمح للنساء بقيادة السيارة,2.0
319,img323.jpg,شاب البنات ولا مرة رح يعرفوا قديش صعب الواحد ي...,4.0
832,img1169.png,داعشي مسلم يقوم بشق عنق كافر اخر,0.0
962,img1310.png,الذي يجعلك تشعر باقصى درجات المحنة,2.0
455,img457.jpg,لجمع الاغراض المدرسية لجمع النقود لجمع العرب,3.0
...,...,...,...
123,img111.jpg,مواطن اردني يهاجر هو ايضا بعدما وصل لوجهته هل ...,2.0
361,img369.jpg,عندما تشعر بالوحدة ثم تشتري آسيويين من الشبكة ...,3.0
704,img1034.jpg,الزنوج البشرة اليبضاء,3.0
248,img268.png,الناس الي بتفكر انه النسوية هي كره للرجال,4.0


# Model

In [ ]:
model_args = ClassificationArgs(
    num_train_epochs=4,
    fp16=False,
    train_batch_size=8,
    learning_rate=1e-5,
    max_seq_length=32,
    save_best_model=True,
    overwrite_output_dir=True,
    save_model_every_epoch=False,
    gradient_accumulation_steps=1,
    labels_list=[0,1,2,3,4]
)

In [ ]:
model = ClassificationModel(
  'bert',
  'UBC-NLP/MARBERT',
  use_cuda=True,
  num_labels=5,
  args=model_args,
) 

In [ ]:
model.train_model(train_df)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/746 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_32_5_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/94 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/94 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/94 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/94 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.


(376, 0.9172032214304868)

In [ ]:
import sklearn

result, model_outputs, wrong_predictions  = model.eval_model(
    test_df,
    acc=sklearn.metrics.accuracy_score,
)

In [ ]:
# model_outputs

In [ ]:
test_df

,images,text,labels
218,img240.jpg,وأخيرا السعودية تسمح للنساء بقيادة السيارة,2.0
319,img323.jpg,شاب البنات ولا مرة رح يعرفوا قديش صعب الواحد ي...,4.0
832,img1169.png,داعشي مسلم يقوم بشق عنق كافر اخر,0.0
962,img1310.png,الذي يجعلك تشعر باقصى درجات المحنة,2.0
455,img457.jpg,لجمع الاغراض المدرسية لجمع النقود لجمع العرب,3.0
...,...,...,...
123,img111.jpg,مواطن اردني يهاجر هو ايضا بعدما وصل لوجهته هل ...,2.0
361,img369.jpg,عندما تشعر بالوحدة ثم تشتري آسيويين من الشبكة ...,3.0
704,img1034.jpg,الزنوج البشرة اليبضاء,3.0
248,img268.png,الناس الي بتفكر انه النسوية هي كره للرجال,4.0


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

_, preds = torch.max(torch.tensor(model_outputs), dim=1)
labels = test_df.labels

print("F1 macro:   {}".format(round(f1_score(labels, preds, average="macro"), 3)))
print("Precision:   {}".format(precision_score(labels,preds, average="macro"), 3))
print("Recall:   {}".format(recall_score(labels,preds, average="macro"), 3))
print("F1 micro:   {}".format(round(f1_score(labels, preds, average="micro"), 3)))